In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
base_path = r"D:\welcome_to_machine_learning\level 3\cnn dataset\my_emotion_dataset"

In [3]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.25,
    zoom_range=[0.7,1.3],
    brightness_range=[0.7,1.3],
    channel_shift_range=20,
    horizontal_flip=True,
    validation_split=0.2
)

In [4]:
train_gen = datagen.flow_from_directory(
    base_path,
    target_size=(224,224),
    batch_size=16,
    class_mode="categorical",
    subset="training"
)

val_gen = datagen.flow_from_directory(
    base_path,
    target_size=(224,224),
    batch_size=16,
    class_mode="categorical",
    subset="validation"
)

num_classes = train_gen.num_classes
print("Detected Classes:", train_gen.class_indices)

Found 1120 images belonging to 7 classes.
Found 280 images belonging to 7 classes.
Detected Classes: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [5]:
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
base_model.trainable = False  # freeze base layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
output = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

In [6]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [7]:
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=3, min_lr=1e-6)
]

In [8]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=callbacks
)

C:\Users\Shivam\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step - accuracy: 0.1553 - loss: 2.2039

C:\Users\Shivam\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


70/70 ━━━━━━━━━━━━━━━━━━━━ 41s 466ms/step - accuracy: 0.1555 - loss: 2.2024 - val_accuracy: 0.2500 - val_loss: 1.8370 - learning_rate: 1.0000e-04
Epoch 2/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 27s 392ms/step - accuracy: 0.2087 - loss: 1.9373 - val_accuracy: 0.4179 - val_loss: 1.6798 - learning_rate: 1.0000e-04
Epoch 3/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 27s 384ms/step - accuracy: 0.2825 - loss: 1.7763 - val_accuracy: 0.4214 - val_loss: 1.6189 - learning_rate: 1.0000e-04
Epoch 4/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 27s 382ms/step - accuracy: 0.3757 - loss: 1.6682 - val_accuracy: 0.4857 - val_loss: 1.5587 - learning_rate: 1.0000e-04
Epoch 5/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 27s 391ms/step - accuracy: 0.3751 - loss: 1.6577 - val_accuracy: 0.5357 - val_loss: 1.5003 - learning_rate: 1.0000e-04
Epoch 6/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 21s 301ms/step - accuracy: 0.3993 - loss: 1.5875 - val_accuracy: 0.5536 - val_loss: 1.4626 - learning_rate: 1.0000e-04
Epoch 7/20
70/70 ━━━━━━━━━━━━━━━━━━━━ 18s 260ms/step - accuracy: 0.46

In [9]:
base_model.trainable = True
for layer in base_model.layers[:-20]: 
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [10]:
history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=callbacks
)

Epoch 1/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 29s 295ms/step - accuracy: 0.3967 - loss: 1.6919 - val_accuracy: 0.7500 - val_loss: 0.9291 - learning_rate: 1.0000e-05
Epoch 2/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 19s 269ms/step - accuracy: 0.5098 - loss: 1.3266 - val_accuracy: 0.6250 - val_loss: 1.0592 - learning_rate: 1.0000e-05
Epoch 3/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 19s 271ms/step - accuracy: 0.5101 - loss: 1.2690 - val_accuracy: 0.7143 - val_loss: 0.9280 - learning_rate: 1.0000e-05
Epoch 4/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 19s 272ms/step - accuracy: 0.5516 - loss: 1.2044 - val_accuracy: 0.6821 - val_loss: 0.9813 - learning_rate: 1.0000e-05
Epoch 5/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 19s 268ms/step - accuracy: 0.5734 - loss: 1.1712 - val_accuracy: 0.7071 - val_loss: 0.9221 - learning_rate: 1.0000e-05
Epoch 6/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 19s 269ms/step - accuracy: 0.6046 - loss: 1.1414 - val_accuracy: 0.7107 - val_loss: 0.9069 - learning_rate: 1.0000e-05
Epoch 7/15
70/70 ━━━━━━━━━━━━━━━━━━━━ 19s 270ms/step - acc

In [11]:
model.save("emotion_transfer.keras")
print("Model Saved ✅")

Model Saved ✅
